In [ ]:
##-- Import libraries
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn import svm
from sklearn.metrics import roc_curve, auc
from scipy import interp
import matplotlib.pyplot as plt

##-- Prepare the feature matrix X from DataFrame columns 'dACC' and 'conflict'
X = np.array([df_I.dACC.ravel(), df_I.conflict.ravel()])
n_features, n_samples  = X.shape

##-- Reshape the feature matrix
X = np.reshape(X,(n_features,n_samples))
X = X.T

##-- Prepare the labels from the DataFrame column 'group'
y = np.array(df_I.group.T)

##-- Initialize function for stratified 6-fold cross-validation
cv = StratifiedKFold(n_splits=6) 

##-- Linear Support Vector Machine (SVM) classifier
classifier = svm.SVC(kernel='linear', probability=True)

##-- Lists to store true positive rates and area-under-curve for each fold
tprs,aucs = [],[]

##-- Array for mean false positive rates across folds
mean_fpr = np.linspace(0, 1, 100)

##-- Initialize plot
figure(num=None, figsize=(10, 8), dpi=80, facecolor='w', edgecolor='k')

##-- Cross-validation and ROC analysis
i = 0
for train, test in cv.split(X, y):
    
    ##-- Train classifier and predict probabilities for test set
    probas_ = classifier.fit(X[train], y[train]).predict_proba(X[test])
    
    ##-- Compute ROC curve and AUC for the current fold
    fpr, tpr, thresholds = roc_curve(y[test], probas_[:, 1])
    tprs.append(interp(mean_fpr, fpr, tpr))
    tprs[-1][0] = 0.0
    roc_auc = auc(fpr, tpr)
    aucs.append(roc_auc)
    
    ##-- Plot ROC curve for the current fold
    plt.plot(fpr, tpr, lw=1, alpha=0.3, label=f'ROC fold {i} (AUC = {roc_auc:.2f})')
    
    i += 1

##-- Plot chance level
plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r', label='Chance', alpha=.8)

##-- Compute + plot mean & std deviation of ROC curves across folds
mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
plt.plot(mean_fpr, mean_tpr, color='b', label=f'Mean ROC (AUC = {mean_auc:.2f} ± {std_auc:.2f})', lw=2, alpha=.8)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
plt.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2, label='± 1 std. dev.')

##-- Label plot
plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic Curve \n(w/ Cross Validation)')
plt.legend(loc="lower right")

##-- Return the plot
plt.show()